# <div align="center">Project 2c: Leakage Assessment<div>

<p><div align="center">Due Date: Monday May 11th, 2020, 11:59 PM</div></p>

<div align="center"><b>You are expected to work in groups of two. Address all questions listed in this document in your final report. Submit the report on <a href="https://mymasonportal.gmu.edu" target="_blank">MyMason</a> in PDF format.</div>

## Test Vector Leakage Assessment
Unprotected cryptographic implementations leak information if not protected properly using side channel counter measures. To test countermeasure effectiveness, key recovery attacks (e.g DPA) can be mounted at several attack points to see if the key can be recovered. 
However, mounting such attacks, which includes analyzing the algorithm to find all possible points of attack, coming up with the power models, obtaining the measurements, etc., is time consuming and requires experience with these attacks and detailed knowledge of the implementation. You have been through this for one possible point of attack of an AES implementation in the previous parts of Project 2.
Leakage assessment methodologies allow you to quickly determine if there is a significat leakage from the implementation.

### Welch's T-Test
Welch's t-test is a moments-based statistical test used in a wide range of scientific research to show if two populations are significantly different. This test is used in the **Test Vector Leakage Assessment (TVLA)** methodology which has been used in many publications to test if there is significant information leakage from an implementation.  
If an implementation is secure against DPA, its power consumption must be independent of the algorithm's intermediate values. This implies that power traces collected when processing fixed data and traces collected when processing random data should be statistically indistinguishable. We call the two trace sets $Q_f$ and $Q_r$ respectively. A $t$ value is calculated as follows:  

\begin{equation}
t = \frac{\mu_f - \mu_r}{\sqrt{\frac{s_f^2}{n_f} + \frac{s_r^2}{n_r}}}     
\end{equation}

Where $\mu_f$ and $\mu_r$ are the means, $s_f$ and $s_r$ are the standard deviations and $n_f$ and $n_r$ are the number of samples in the sets. 
The **null hypothesis** is that the means of the two trace sets $Q_f$ and $Q_r$ are equal (i.e.,  the two trace sets are indistinguishable). We use the calculated $t$ value as an indicator to reject the null hypothesis at certain confidence level. If $\mid t \mid > 4.5 $ we reject the null hypothesis at a confidence level of $99.999\%$ (i.e., $p<10^{-5}$). This means the two sets $Q_f$ and $Q_r$ are distinguishable and the cryptographic core is likely leaking information. However, this doesn't prove that the leakage can be exploited to obtain secret information, and it doesn't recover any secret information itself.

The point of doing such a test is to give the countermeasure designer more confidence on the soundness of the protection. However, recent research has shown that even implementations passing the t-test can leak information in ways the t-test does not capture. More advanced test have been therfore developed. Never the less, TVLA is still a powerful tool and the first test to be performed for SCA leakage evaluation.

## TVLA on Unprotected AES

Run the following cell to load the needed libraries.

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import fobos.traceset as traceset
import fobos.projmgr as projmgr
import fobos.ttest

### AES Unprotected

First, you will apply TVLA to the unprotected AES-128 implementation you already perfromed SCA on and document your observations. The traces were collected using the FOBOS acquisition module as follows:

1. Test vectors were generated. The test vectors are composed of random test vectors and a fixed test vectors.
The vectors are randomly interleaved. You can think of this as throwing a coin and based on the outcoume, decide whether the next test vector should be randomly generated or the fixed vector should be used. A meta file that keeps track of fixed vs random test vectors is generated. The file is called **fvrchoicefile.txt**.

2. The acquisition setting are configured and saved in the acquisitionConfi.txt file.
 
3. The FOBOS acquistion module was run to collect traces from the AES-128 instantiated on a Xilinx Spartan-6 FPGA and a power power probe was used to measure power consumption.

4. The project data is saved at fobosworkspace/aes_ttest/.

<div class="alert alert-block alert-info">
    
#### Questions:

We will first work with the data in fobosworkspace/aes_ttest/capture/attempt-01

1. Looking at the acquistionConfig.json file, summarize in words the settings used.
2. Inspect the first five blocks in the plaintext.txt file and note which are randomly generated and which are the fixed block.
3. Look at the first five entries in the fvrchoicefile.txt (0's and 1's). What do they mean?
</div>


Run the following cell to configure the input and output files locations.

In [ ]:
# Configure project directories ################################
PROJECT_NAME1 = "attempt-01"
WORKSPACE1 = "fobosworkspace/aes_ttest/capture"
pm = projmgr.ProjectManager()
pm.setWorkSpaceDir(WORKSPACE1)
pm.setProjName(PROJECT_NAME1)
projDir1 = pm.getProjDir()
analysisDir1 = pm.getAnalysisDir()
TRACES_FILE1 = os.path.join(projDir1, 'powerTraces.npy')
FVR_FILE1 = os.path.join(projDir1, 'fvrchoicefile.txt')

<span style="color:red">Please pay attention to the messages generated by the previous cell and note the attempt-xx directory number.</span>

Run the following cell to display some of the traces collected form FOBOS running the unproteced AES.

In [ ]:
# configure plotting settings
tConf = {}
##t-test/plotting settings
tConf['plot'] = True
tConf['plotSize'] = (10,8)
tConf['plotFontSize'] = 18
MAX_TRACES = 100
#Read acquisition config
acqConf = {}
configFile = open(os.path.join(projDir1, 'acquisitionConfig.json'))
acqConf = json.load(configFile)
print(f'Acquisition config = {acqConf}')
# load traces from file.
traceSetOrig = traceset.TraceSet(traceNum=MAX_TRACES,
                                 fileName=TRACES_FILE1)
measuredPowerOrig = traceSetOrig.traces
print(f'The shape of the traces matrix is {measuredPowerOrig.shape}')
plt.figure(figsize=tConf['plotSize'])
plt.rcParams.update({'font.size': tConf['plotFontSize']})
plt.xlabel('Sample')
plt.ylabel('Amplitdue')
plt.title('Traces')
for i in range(5):
    plt.plot(measuredPowerOrig[i])
    
plt.savefig(os.path.join(analysisDir1, 'traces.png'))


<div class="alert alert-block alert-info">
    
#### Questions:
4. Include the graph above in your report
</div>


Run the following code to perform the t-test. 
The traces are already split traces into two sets: the one generated using random vectors and the other using fixed vectors.

The plot has sample numbers on the x-axis and t-values on the y-axis.
Two lines at 4.5 and -4.5 indicate the threshold discussed above.

The **step** parameter in the code below allows the input files to be loaded in small chunks to avoid filling the memory.

In [ ]:
tConf['traceNum'] = 20     # number of traces to analyze, default: 20
tConf['plotSize'] = (10,8)
tConf['plotFontSize'] = 18
traceFile0 = os.path.join(projDir1, 'trace_set0_50MSps.npy')
traceFile1 = os.path.join(projDir1, 'trace_set1_50MSps.npy')
plotFile = os.path.join(analysisDir1, 't-test-result.png')
test = fobos.ttest.Ttest()
tvals = test.doTtest(traceFile0=traceFile0,
                  traceFile1=traceFile1, 
                  numTraces=tConf['traceNum'],
                  step='auto',
                  analysisDir=analysisDir1)
test.plotTValues(t=tvals, 
                 startXlim=0, 
                 endXlim='All', 
                 startYlim=-50, 
                 endYlim=50, 
                 analysisDir=analysisDir1,
                 traceNum=tConf['traceNum'],
                 plotSize=tConf['plotSize'],
                 plotFontSize=tConf['plotFontSize']
                )

### The effect of trace number

In this question, you will see the effect of running the t-test using less traces.


<div class="alert alert-block alert-info">
    
#### Questions:
5.  Include the graph above for 20 traces in your report. 
6.  What do you observe about the result, does this implementation about AES pass the t-test?
7.  Now set the trace number to 200 and run the code again. Include the resulting graph in your report.
8.  Now set the trace number to 2000 and run the code again. Include the resulting graph in your report.
9.  What differences do you oberve in the three graphs?
10. Compare the last graph to the power traces you plotted above. Can you observe the AES clock cycles?
</div>

### The effect of sampling frequency

In this section, we will load another set of AES traces collected using the same settings as the previous data. However, the sampling rate was reduced from 50 M Sample/sec to 5 M Sample/sec.

The trace data is stored in fobosworkspace/aes_ttest/capture/attempt-02

Run the following cell to load and plot the traces:


In [ ]:
# Configure project directories ################################
PROJECT_NAME2 = "attempt-02" # the location is likely in the form attampt-i (e.g. attempt-1)
WORKSPACE2 = "fobosworkspace/aes_ttest/capture"
pm = projmgr.ProjectManager()
pm.setWorkSpaceDir(WORKSPACE2)
pm.setProjName(PROJECT_NAME2)
projDir2 = pm.getProjDir()
analysisDir2 = pm.getAnalysisDir()
TRACES_FILE2 = os.path.join(projDir2, 'powerTraces.npy')
FVR_FILE2 = os.path.join(projDir2, 'fvrchoicefile.txt')
######################################
tConf = {}
##t-test/plotting settings
tConf['plot'] = True
tConf['plotSize'] = (10,8)
tConf['plotFontSize'] = 18
MAX_TRACES = 100
#Read acquisition config
acqConf = {}
configFile = open(os.path.join(projDir2, 'acquisitionConfig.json'))
acqConf = json.load(configFile)
print(f'Acquisition config = {acqConf}')
# load traces from file.
traceSetOrig = traceset.TraceSet(traceNum=MAX_TRACES,
                            fileName=TRACES_FILE2)
measuredPowerOrig = traceSetOrig.traces
print(f'The shape of the traces matrix is {measuredPowerOrig.shape}')
plt.figure(figsize=tConf['plotSize'])
plt.rcParams.update({'font.size': tConf['plotFontSize']})
plt.xlabel('Sample')
plt.ylabel('Amplitdue')
plt.title('Traces')
for i in range(10,15):
    plt.plot(measuredPowerOrig[i])
plt.savefig(os.path.join(analysisDir2, 'traces.png'))

<div class="alert alert-block alert-info">
    
#### Questions:
11. Include the graph above in your report.
12. What is the difference between the traces with the two different sampling rates?
</div>

Run the following cell to perform a t-test using the traces above 

In [ ]:
tConf['traceNum'] = 20           # number of traces to analyze, default: 20
tConf['plotSize'] = (10,8)
tConf['plotFontSize'] = 18
traceFile0 = os.path.join(projDir2, 'trace_set0_5MSps.npy')
traceFile1 = os.path.join(projDir2, 'trace_set1_5MSps.npy')
plotFile = os.path.join(analysisDir2, 't-test-result.png')
test = fobos.ttest.Ttest()
tvals = test.doTtest(traceFile0=traceFile0,
                  traceFile1=traceFile1, 
                  numTraces=tConf['traceNum'],
                  step='auto', 
                  analysisDir=analysisDir2)
test.plotTValues(t=tvals, 
                 startXlim=0, 
                 endXlim='All', 
                 startYlim=-50, 
                 endYlim=50,
                 traceNum=tConf['traceNum'],
                 analysisDir=analysisDir2,
                 plotSize=tConf['plotSize'],
                 plotFontSize=tConf['plotFontSize']
                )

<div class="alert alert-block alert-info">
    
#### Questions:
13.  Include the graph above for 20 traces in your report. 
14.  What do you observe about the result, does this implementation about AES pass the t-test?
15.  Now set the trace number to 200 and run the code again. Include the resulting graph in your report.
16.  Now set the trace number to 2000 and run the code again. Include the resulting graph in your report.
17.  What is the effect of reducing sampling frequency? (Note: compare to t-test with the same number of traces)
</div>


## TVLA on Unprotected Ascon


Ascon is an authenticated cipher that is currently a candidate in NIST Lightweight Cryptography (LWC) standardization effort.
Below, we perform a t-test on an unprotected FPGA implementation of Ascon.

Run the following cell to configure the input and output files locations.

In [ ]:
# Configure project directories ################################
PROJECT_NAME3 = "attempt-1" # the location is likely in the form attampt-i (e.g. attempt-1)
WORKSPACE3 = "fobosworkspace/ascon_unprotected/capture"
pm = projmgr.ProjectManager()
pm.setWorkSpaceDir(WORKSPACE3)
pm.setProjName(PROJECT_NAME3)
projDir3 = pm.getProjDir()
analysisDir3 = pm.getAnalysisDir()
TRACES_FILE3 = os.path.join(projDir3, 'powerTraces.npy')
FVR_FILE3 = os.path.join(projDir3, 'fvrchoicefile.txt')

Run the following cell to display some of the traces collected form FOBOS running the unproteced Ascon.

In [ ]:
##plotting settings
tConf['plot'] = True
tConf['plotSize'] = (10,8)
tConf['plotFontSize'] = 18
tConf['traceNum'] = 2000
MAX_TRACES = 10
# load traces from file.
traceSetOrig = traceset.TraceSet(traceNum=MAX_TRACES,
                            fileName=TRACES_FILE3)
measuredPowerOrig = traceSetOrig.traces
print(f'The shape of the traces matrix is {measuredPowerOrig.shape}')
plt.figure(figsize=tConf['plotSize'])
plt.rcParams.update({'font.size': tConf['plotFontSize']})
plt.xlabel('Sample')
plt.ylabel('Amplitdue')
plt.title('Traces')
for i in range(5):
    plt.plot(measuredPowerOrig[i])
plt.savefig(os.path.join(analysisDir3, 'traces.png'))

Run the t-test on the two trace sets using the following code.

In [ ]:
tConf['traceNum'] = 2000         # number of traces to analyze, default: 2000
tConf['plotSize'] = (10,8)
tConf['plotFontSize'] = 18
traceFile0 = os.path.join(projDir3, 'trace_set0.npy')
traceFile1 = os.path.join(projDir3, 'trace_set1.npy')
plotFile = os.path.join(analysisDir3, 't-test-result.png')
test = fobos.ttest.Ttest()
tvals = test.doTtest(traceFile0=traceFile0,
                  traceFile1=traceFile1, 
                  numTraces=tConf['traceNum'],
                  step='auto', 
                  analysisDir=analysisDir3)
test.plotTValues(t=tvals, 
                 startXlim=0, 
                 endXlim='All', 
                 startYlim=-10, 
                 endYlim=10,
                 traceNum=tConf['traceNum'],
                 analysisDir=analysisDir3,
                 plotSize=tConf['plotSize'],
                 plotFontSize=tConf['plotFontSize']
                )

<div class="alert alert-block alert-info">
    
#### Questions:
18.  Include the graph above in your report.
19.  What are the approximate sample numbers where the t-test shows information leakage?
20.  Repeat the test above with less traces, what is the largest number of traces that yields a **passing** test?
</div>


## TVLA on Protected Ascon 

Below, we perform a T test on an FPGA implemetation of Ascon that is protected against SCA attackes using the Threshold Implementation (TI) technique.
TI is a form of masking where sensitive data is split into shares. To reconstruct the sensitive value, all shares are needed. The TI implementation was designed to withstand first order attacks.

Run the following cell to configure the input and output files locations.

In [ ]:
# Configure project directories ################################
PROJECT_NAME4 = "attempt-1" # the location is likely in the form attampt-i (e.g. attempt-1)
WORKSPACE4 = "fobosworkspace/ascon_protected/capture"
pm = projmgr.ProjectManager()
pm.setWorkSpaceDir(WORKSPACE4)
pm.setProjName(PROJECT_NAME4)
projDir4 = pm.getProjDir()
analysisDir4 = pm.getAnalysisDir()
######
TRACES_FILE4 = os.path.join(projDir4, 'powerTraces.npy')
FVR_FILE4 = os.path.join(projDir4, 'fvrchoicefile.txt')

Run the following cell to display some of the traces collected form FOBOS running the proteced Ascon.

In [ ]:
##plotting settings
tConf['plot'] = True
tConf['plotSize'] = (10,8)
tConf['plotFontSize'] = 18
tConf['traceNum'] = 2000
MAX_TRACES = 10
# load traces from file.
traceSetOrig = traceset.TraceSet(traceNum=MAX_TRACES,
                            fileName=TRACES_FILE4)
measuredPowerOrig = traceSetOrig.traces
print(f'The shape of the traces matrix is {measuredPowerOrig.shape}')
plt.figure(figsize=tConf['plotSize'])
plt.rcParams.update({'font.size': tConf['plotFontSize']})
plt.xlabel('Sample')
plt.ylabel('Amplitdue')
plt.title('Traces')
for i in range(5):
    plt.plot(measuredPowerOrig[i])
plt.savefig(os.path.join(analysisDir4, 'traces.png'))

Run the t-test on the two trace sets using the following code.

In [ ]:
tConf['traceNum'] = 2000             # number of traces to analyze, default: 2000
tConf['plotSize'] = (10,8)
tConf['plotFontSize'] = 18
traceFile0 = os.path.join(projDir4, 'trace_set0.npy')
traceFile1 = os.path.join(projDir4, 'trace_set1.npy')
plotFile = os.path.join(analysisDir4, 't-test-result.png')
test = fobos.ttest.Ttest()
tvals = test.doTtest(traceFile0=traceFile0,
                  traceFile1=traceFile1, 
                  numTraces=tConf['traceNum'],
                  step='auto', 
                  analysisDir=analysisDir4)
test.plotTValues(t=tvals, 
                 startXlim=0, 
                 endXlim='All', 
                 startYlim=-10, 
                 endYlim=10,
                 traceNum=tConf['traceNum'],
                 analysisDir=analysisDir4,
                 plotSize=tConf['plotSize'],
                 plotFontSize=tConf['plotFontSize']
                )

<div class="alert alert-block alert-info">
    
#### Questions:
21.  Copy the graph into your report.
22.  What is the result of the t-test of the protected implementation of Ascon?
</div>
